In [1]:
import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
import matplotlib.pyplot as plt # library for displaying graphs
import json # library to handle JSON files
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Q1 - Extracting Data From Wikipedia

In [2]:
import requests # library to handle requests
from bs4 import BeautifulSoup # library to retrieve data from a website

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_data = soup.find('div', class_='mw-parser-output')
table = table_data.table.tbody

columns = ['PostalCode', 'Borough', 'Neighborhood']
data = dict({key:[]*len(columns) for key in columns})

for line in table.find_all('tr'):
    for i,column in zip(line.find_all('td'),columns):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

# Converting Dictionary to DataFrame

In [4]:
df = pd.DataFrame.from_dict(data=data)[columns] # converts dict to dataframe
print(df.shape)
df.head()

(287, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


# Only Processing the cells that have an assigned Borough

In [5]:
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.shape

(210, 3)

# Arranging More Than One Neighborhood

In [6]:
df_new = df.groupby(['PostalCode','Borough'], sort=False).agg( ', '.join).reset_index()
df_new.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Not assigned
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


# Arranging Not assigned Neighbourhoods

In [7]:
df_new.loc[df_new['Neighborhood'] =='Not assigned' , 'Neighborhood'] = df_new['Borough']

In [8]:
df_new.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [9]:
df_new.shape

(103, 3)

# Q2 - Adding Lat and Long values using csv file

In [10]:
coordinates = pd.read_csv('GC.csv')
coordinates.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [11]:
df_new_sorted = df_new.sort_values('PostalCode')
df_new_sorted.reset_index(inplace=True)
df_new_sorted = df_new_sorted.drop('index',1)
df_new_sorted.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
df_new_sorted['Latitude'] = coordinates['Latitude']
df_new_sorted['Longitude'] = coordinates['Longitude']

In [13]:
df_new_sorted.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Q3

## Explore and cluster the neighborhoods in Toronto¶


### Use geopy library to get the latitude and longitude values of Toronto.¶


In [14]:
toronto_data=df_new_sorted[df_new_sorted['Borough'].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [15]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [16]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [17]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new_sorted['Latitude'], df_new_sorted['Longitude'], df_new_sorted['Borough'], df_new_sorted['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# FourSquare

In [19]:
CLIENT_ID = 'PG5AZBDTWPLIPWHBBEVIWWMESSPEV123MQTEQ5FLNNCLDA5F' 
CLIENT_SECRET = 'GVRBNT2QVABD1F0EWP5QSQ5H0NQLLAKLWGW1S1PJXKEVWWXN' 
VERSION = '20180604'

In [20]:
radius = 500
LIMIT = 100

In [21]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PG5AZBDTWPLIPWHBBEVIWWMESSPEV123MQTEQ5FLNNCLDA5F&client_secret=GVRBNT2QVABD1F0EWP5QSQ5H0NQLLAKLWGW1S1PJXKEVWWXN&ll=43.653963,-79.387207&v=20180604&radius=500&limit=100'

In [22]:
results = requests.get(url).json()

In [23]:
results

{'meta': {'code': 200, 'requestId': '5e38658d0be7b4001bb51824'},
 'response': {'venues': [{'id': '4f04779a02d5cce0cfc06151',
    'name': 'Chinese Visa Application Service Center',
    'location': {'address': '393 University Ave, Suite 1501',
     'crossStreet': 'in University Centre',
     'lat': 43.65402839343005,
     'lng': -79.38736458967021,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65402839343005,
       'lng': -79.38736458967021}],
     'distance': 14,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['393 University Ave, Suite 1501 (in University Centre)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d126941735',
      'name': 'Government Building',
      'pluralName': 'Government Buildings',
      'shortName': 'Government',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
       'suffix': '.png'},
      'primary': True}],
 

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [24]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,venuePage.id,location.neighborhood
0,4f04779a02d5cce0cfc06151,Chinese Visa Application Service Center,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1580754317,False,"393 University Ave, Suite 1501",in University Centre,43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",14,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",NaN,NaN,NaN
1,4c655857f67ec928e7485727,393 University Ave,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1580754317,False,393 University Ave,NaN,43.653752,-79.388150,"[{'label': 'display', 'lat': 43.65375248805427...",79,CA,Toronto,ON,Canada,"[393 University Ave, Toronto ON M5G 1E6, Canada]",M5G 1E6,NaN,NaN
2,4b66e19ff964a520922e2be3,Ontario Superior Court of Justice,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",v-1580754317,False,393 University Avenue,NaN,43.653521,-79.387193,"[{'label': 'display', 'lat': 43.6535208759085,...",49,CA,Toronto,ON,Canada,"[393 University Avenue, Toronto ON, Canada]",NaN,NaN,NaN
3,4d8a3465d85f370415cbf8db,Ontario Superior Court House,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",v-1580754317,False,361 University Ave,at Queen St W,43.653764,-79.385925,"[{'label': 'display', 'lat': 43.65376404494388...",105,CA,Toronto,ON,Canada,"[361 University Ave (at Queen St W), Toronto O...",M5G 1T3,NaN,NaN
4,4b8c0d41f964a520e5ba32e3,Marcello's Market and Deli,"[{'id': '4bf58dd8d48988d146941735', 'name': 'D...",v-1580754317,False,393 University Ave.,University Centre,43.653761,-79.388098,"[{'label': 'display', 'lat': 43.65376090619767...",75,CA,Toronto,ON,Canada,"[393 University Ave. (University Centre), Toro...",M5G 1E6,NaN,NaN


#### Define information of interest and filter dataframe

In [25]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,neighborhood,id
0,Chinese Visa Application Service Center,Government Building,"393 University Ave, Suite 1501",in University Centre,43.654028,-79.387365,"[{'label': 'display', 'lat': 43.65402839343005...",14,CA,Toronto,ON,Canada,"[393 University Ave, Suite 1501 (in University...",NaN,NaN,4f04779a02d5cce0cfc06151
1,393 University Ave,Office,393 University Ave,NaN,43.653752,-79.388150,"[{'label': 'display', 'lat': 43.65375248805427...",79,CA,Toronto,ON,Canada,"[393 University Ave, Toronto ON M5G 1E6, Canada]",M5G 1E6,NaN,4c655857f67ec928e7485727
2,Ontario Superior Court of Justice,Courthouse,393 University Avenue,NaN,43.653521,-79.387193,"[{'label': 'display', 'lat': 43.6535208759085,...",49,CA,Toronto,ON,Canada,"[393 University Avenue, Toronto ON, Canada]",NaN,NaN,4b66e19ff964a520922e2be3
3,Ontario Superior Court House,Courthouse,361 University Ave,at Queen St W,43.653764,-79.385925,"[{'label': 'display', 'lat': 43.65376404494388...",105,CA,Toronto,ON,Canada,"[361 University Ave (at Queen St W), Toronto O...",M5G 1T3,NaN,4d8a3465d85f370415cbf8db
4,Marcello's Market and Deli,Deli / Bodega,393 University Ave.,University Centre,43.653761,-79.388098,"[{'label': 'display', 'lat': 43.65376090619767...",75,CA,Toronto,ON,Canada,"[393 University Ave. (University Centre), Toro...",M5G 1E6,NaN,4b8c0d41f964a520e5ba32e3
5,375 University Avenue,Building,375 University Avenue,NaN,43.653265,-79.387640,"[{'label': 'display', 'lat': 43.65326499063437...",85,CA,Toronto,ON,Canada,"[375 University Avenue, Toronto ON, Canada]",NaN,NaN,4c6980cd35d3be9a107d1f06
6,Court File Storage Room,None,"393 University Avenue, Suite 1003",NaN,43.654091,-79.386154,"[{'label': 'display', 'lat': 43.65409100564799...",86,CA,Toronto,ON,Canada,"[393 University Avenue, Suite 1003, Toronto ON...",M5G,NaN,4c6eb4e5d5c3a1cd2e10c92b
7,DoubleTree by Hilton,Hotel,108 Chestnut Street,Dundas St W,43.654608,-79.385942,"[{'label': 'display', 'lat': 43.6546083, 'lng'...",124,CA,Toronto,ON,Canada,"[108 Chestnut Street (Dundas St W), Toronto ON...",M5G 1R3,NaN,52ce14b0498e50457ce11780
8,Farmers Market,Farmers Market,100 Queen Street West,NaN,43.652946,-79.383581,"[{'label': 'display', 'lat': 43.65294599924012...",313,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,NaN,4c50577d3a6f2d7ff1cc6877
9,Staples,Paper / Office Supplies Store,375 University Avenue,at Armoury St,43.653502,-79.387240,"[{'label': 'display', 'lat': 43.6535023, 'lng'...",51,CA,Toronto,ON,Canada,"[375 University Avenue (at Armoury St), Toront...",M5G 2J5,NaN,4b0ade68f964a520302923e3


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### New dataframe called *toronto_venues*.

In [27]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [28]:
print(toronto_venues.shape)
toronto_venues.head()

(1714, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


#### How many venues were returned for each neighborhood

In [29]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton, Exhibition Place, Parkdale Village",23,23,23,23,23,23
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",47,47,47,47,47,47
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",87,87,87,87,87,87
Christie,19,19,19,19,19,19


## 3. Analyze Each Neighborhood

In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True) 
toronto_onehot.insert(loc=0, column='Neighborhood', value=toronto_venues['Neighborhood'] )
toronto_onehot.shape

(1714, 230)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.02,0.0,0.0,0.010000,0.0,0.03,0.0,0.0000,0.000000,0.0,0.000000,0.030000,0.0,0.030000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0000,0.02,0.0000,0.01,0.030000,0.0000,0.0,0.01,0.03,0.010000,0.0,0.000000,0.040000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.070000,0.0,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.01,0.01,0.010000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0,0.000000,0.0000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.01,0.0,0.000000,0.000000,0.0,0.0,0.0,0.010000,0.0,0.0000,0.0000,0.0,0.02,0.0,0.0,0.01,0.0,0.01,0.01,0.000000,0.000000,0.000000,0.020000

In [32]:
toronto_grouped.shape

(39, 230)

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Restaurant,Breakfast Spot,Bakery,Bar,Thai Restaurant,Asian Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Gourmet Shop
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Nightclub,Café,Coffee Shop,Yoga Studio,Gym,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,Business Reply Mail Processing Centre 969 Eastern,Garden,Skate Park,Auto Workshop,Farmers Market,Light Rail Station,Smoke Shop,Burrito Place,Pizza Place,Spa,Fast Food Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


### Making Clusters for Neighborhood¶


In [35]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [36]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Asian Restaurant,Pub,Trail,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Park,Pub,Sandwich Place,Light Rail Station,Burger Joint,Burrito Place,Italian Restaurant,Liquor Store,Fast Food Restaurant,Ice Cream Shop
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [37]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide, King, Richmond",Coffee Shop,Steakhouse,Café,Restaurant,Breakfast Spot,Bakery,Bar,Thai Restaurant,Asian Restaurant,Burger Joint
1,0,Berczy Park,Coffee Shop,Cocktail Bar,Steakhouse,Café,Cheese Shop,Seafood Restaurant,Beer Bar,Bakery,Farmers Market,Gourmet Shop
2,0,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Nightclub,Café,Coffee Shop,Yoga Studio,Gym,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,0,Business Reply Mail Processing Centre 969 Eastern,Garden,Skate Park,Auto Workshop,Farmers Market,Light Rail Station,Smoke Shop,Burrito Place,Pizza Place,Spa,Fast Food Restaurant
4,0,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Sculpture Garden


In [38]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [39]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Clusters

### Cluster 1

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Health Food Store,Asian Restaurant,Pub,Trail,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Bookstore,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
42,East Toronto,0,Park,Pub,Sandwich Place,Light Rail Station,Burger Joint,Burrito Place,Italian Restaurant,Liquor Store,Fast Food Restaurant,Ice Cream Shop
43,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Brewery,Italian Restaurant,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place
45,Central Toronto,0,Park,Dance Studio,Breakfast Spot,Sandwich Place,Food & Drink Shop,Department Store,Hotel,Gym,Convenience Store,Cosmetics Shop
46,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Bagel Shop,Salon / Barbershop,Diner,Spa,Fast Food Restaurant,Sporting Goods Shop,Restaurant
47,Central Toronto,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Brewery
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Light Rail Station,Vietnamese Restaurant,Supermarket,Liquor Store,Sushi Restaurant,Sports Bar,American Restaurant
51,Downtown Toronto,0,Coffee Shop,Café,Bakery,Restaurant,Pub,Park,Italian Restaurant,Pizza Place,Market,Chinese Restaurant
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Gym


### Cluster 2

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Tennis Court,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 3

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,2,Park,Playground,Trail,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


### Cluster 4

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Home Service,Garden,Ice Cream Shop,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### Cluster 5

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Bus Line,Swim School,Dessert Shop,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
